In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import scipy
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy import stats
from sklearn.model_selection import StratifiedKFold
from __future__ import division

In [56]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:20,.2f}'.format

In [212]:
train = pd.read_csv("C:\\Users\\Pavan\\Desktop\\Avidya\\churn_prediction\\train.csv")

C:\Users\Pavan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [183]:

def get_missing_and_present_cols(df):
    total_columns = df.columns
    
    for column in total_columns:
        if df[column].isnull().sum() == 0:
            non_missing_val_cols.append(column)

    print("There are "+str(len(non_missing_val_cols)) +" columns without any missing values")
    
    
    for column in total_columns:
        if df[column].isnull().sum() > 0:
            missing_val_cols.append(column)
    print("There are "+str(len(missing_val_cols)) +" columns with missing values")
    
    
    for column in non_missing_val_cols:
        if df[column].dtype == "object":
            non_missing_cat_cols.append(column)
    print("There are "+str(len(non_missing_cat_cols)) +" categorical columns without any missing values")
    
    
    for col in non_missing_val_cols:
        if df[col].dtype == "int64" or df[col].dtype == "float64" :
            non_missing_cont_cols.append(col)
    print("There are "+str(len(non_missing_cont_cols)) +" continuous columns without missing values")
    
    
    for column in missing_val_cols:
        if df[column].dtype == "object":
            missing_cat_cols.append(column)
    print("There are "+str(len(missing_cat_cols)) +" categorical columns with missing values")
    
    
    cols= ["Req_Resolved_PrevQ1","Query_Resolved_PrevQ1","Complaint_Resolved_PrevQ1"]
    for col in missing_val_cols:
        if df[col].dtype == "int64" or df[col].dtype == "float64" and col not in cols :
            missing_cont_cols.append(col)
    print ("There are "+str(len(missing_cont_cols)) +" continuous columns with missing values")
    

In [184]:
def add_missing_val_count(df):
    df['MISSING_VALUE_COUNT']=df.isnull().sum(axis = 1)
    df['INV_MISSING_VALUE_COUNT'] = df.notnull().sum(axis = 1)
    return ["INV_MISSING_VALUE_COUNT","MISSING_VALUE_COUNT"]

In [185]:
def add_missing_val_cat_cols(df, cols):
    for col in cols:
        print(col +" : "+str(df[col].isnull().sum()))
        df.loc[df[col].isnull(),col] ="MISSING_VAL"

In [186]:
def encode_cat(df, cols,suffix ="EN"):
    encoded_cols = []
    le = preprocessing.LabelEncoder()
    for col in cols:
        print("processing "+col)
        df.loc[:,col+suffix] = le.fit_transform(df[col])
        encoded_cols.append(col+suffix)
    return encoded_cols

In [187]:
def treat_complaints(df):
    cols= ["Req_Resolved_PrevQ1","Query_Resolved_PrevQ1","Complaint_Resolved_PrevQ1"]
    for col in cols:
        df.loc[df[col]==">",col] = 0
        df.loc[df[col].isnull(),col] = 0
        df[col] = df[col].astype(float)
        print(df[col].value_counts())  
    

In [229]:

def feature_importance( clf,temp_cols_list,del_useless=False):
    if del_useless:
        del useless_cols[:]
    #useless_cols = []
    important_features_dict = {}
    for x,i in enumerate(clf.feature_importances_):
    #print str(x) + " "+str(i ) +" "+ cols_1_list[x]
        important_features_dict[temp_cols_list[x]]=i
        important_features_list = sorted(important_features_dict,
                                     key=important_features_dict.get,
                                     reverse=True)

    for i in important_features_list:
        val =""
        #if i in missing_cat_cols:
        #    val = "***"
        if(important_features_dict[i] < 0.00001):
            useless_cols.append(i)
        print(str(important_features_dict[i]) + " " + i + " "+val)

In [476]:
def predict_and_save_file(mod,df,cols,target="Responders",id_col="UCIC_ID",prob = True):
    if(prob):
        df[target] = mod.predict_proba(df.loc[:,cols])[:,1]
    else:
        df[target] = mod.predict(df.loc[:,cols])
    submission = df.loc[:,[id_col,target]]
    submission.sort(columns =target,ascending=False, inplace=True)
    submission.to_csv("C:\\Users\\Pavan\\Desktop\\Avidya\\churn_prediction\\submission"+str(np.random.randint(1000))+".csv",index=False)


In [200]:
def calculate_lift(df, id_col ='UCIC_ID' ,output_col='Responders',predicted_col='predicted'):
    #sorted_predicted.reindex()
    total_20 = df.shape[0]*0.2
    sorted_predicted =  df.sort(predicted_col,ascending = False).head(int(total_20))
    sorted_predicted_inv = df.sort(predicted_col,ascending = True).head(int(df.shape[0]-total_20))
    #print sorted_predicted.loc[total_20-3:total_20,:].head()
    print("total_20 "+str(total_20))
    top_20_responders = sorted_predicted[output_col].sum()
    print("top_20_responders "+str(top_20_responders))
    tot_responders=df[output_col].sum()
    print("tot_responders "+str(tot_responders))
    lift = top_20_responders/tot_responders
    print("LIFT----- "+str(lift)+"  -------")
    return lift

In [213]:
non_missing_val_cols = []
missing_val_cols = []
non_missing_cat_cols = []
non_missing_cont_cols =[]
missing_cat_cols = []
missing_cont_cols = []
get_missing_and_present_cols(train)

for col in missing_cat_cols:
    if col in ["Req_Resolved_PrevQ1","Query_Resolved_PrevQ1","Complaint_Resolved_PrevQ1"]:
        missing_cat_cols.remove(col)
for col in missing_cat_cols:
    if col in ["Req_Resolved_PrevQ1","Query_Resolved_PrevQ1","Complaint_Resolved_PrevQ1"]:
        missing_cat_cols.remove(col)

There are 206 columns without any missing values
There are 171 columns with missing values
There are 8 categorical columns without any missing values
There are 198 continuous columns without missing values
There are 30 categorical columns with missing values
There are 141 continuous columns with missing values


In [214]:
missing_cat_cols

['EMAIL_UNSUBSCRIBE',
 'OCCUP_ALL_NEW',
 'city',
 'FINAL_WORTH_prev1',
 'ENGAGEMENT_TAG_prev1',
 'AGRI_TAG_LIVE',
 'AL_CNC_TAG_LIVE',
 'AL_TAG_LIVE',
 'BL_TAG_LIVE',
 'CC_TAG_LIVE',
 'CE_TAG_LIVE',
 'CV_TAG_LIVE',
 'DEMAT_TAG_LIVE',
 'EDU_TAG_LIVE',
 'GL_TAG_LIVE',
 'HL_TAG_LIVE',
 'SEC_ACC_TAG_LIVE',
 'INS_TAG_LIVE',
 'LAS_TAG_LIVE',
 'MF_TAG_LIVE',
 'OTHER_LOANS_TAG_LIVE',
 'PL_TAG_LIVE',
 'RD_TAG_LIVE',
 'FD_TAG_LIVE',
 'TL_TAG_LIVE',
 'TWL_TAG_LIVE',
 'lap_tag_live']

In [215]:
add_missing_val_count(train)
train.head()
add_missing_val_cat_cols(train,missing_cat_cols)
encode_cat(train,non_missing_cat_cols)
encode_cat(train,missing_cat_cols,suffix="EN_MIS")
treat_complaints(train)
cols= ["Req_Resolved_PrevQ1","Query_Resolved_PrevQ1","Complaint_Resolved_PrevQ1"]


EMAIL_UNSUBSCRIBE : 286003
OCCUP_ALL_NEW : 96
city : 5701
FINAL_WORTH_prev1 : 186
ENGAGEMENT_TAG_prev1 : 186
AGRI_TAG_LIVE : 299980
AL_CNC_TAG_LIVE : 298286
AL_TAG_LIVE : 285215
BL_TAG_LIVE : 297866
CC_TAG_LIVE : 198127
CE_TAG_LIVE : 299859
CV_TAG_LIVE : 299328
DEMAT_TAG_LIVE : 238386
EDU_TAG_LIVE : 299989
GL_TAG_LIVE : 297821
HL_TAG_LIVE : 293623
SEC_ACC_TAG_LIVE : 243394
INS_TAG_LIVE : 259575
LAS_TAG_LIVE : 299999
MF_TAG_LIVE : 272500
OTHER_LOANS_TAG_LIVE : 296724
PL_TAG_LIVE : 297413
RD_TAG_LIVE : 275370
FD_TAG_LIVE : 199477
TL_TAG_LIVE : 299955
TWL_TAG_LIVE : 298738
lap_tag_live : 299242
processing HNW_CATEGORY
processing EFT_SELF_TRANSFER_PrevQ1
processing Billpay_Active_PrevQ1_N
processing Billpay_Reg_ason_Prev1_N
processing Charges_cnt_PrevQ1_N
processing FRX_PrevQ1_N
processing RBI_Class_Audit
processing gender_bin
processing EMAIL_UNSUBSCRIBE
processing OCCUP_ALL_NEW
processing city
processing FINAL_WORTH_prev1
processing ENGAGEMENT_TAG_prev1
processing AGRI_TAG_LIVE
processin

In [216]:
fix_goddamn_hell(train)

['C_prev1', 'C_prev2', 'C_prev3', 'C_prev4', 'C_prev5', 'C_prev6']
['C_prev1_NEW', 'C_prev2_NEW', 'C_prev3_NEW', 'C_prev4_NEW', 'C_prev5_NEW', 'C_prev6_NEW']
['D_prev1', 'D_prev2', 'D_prev3', 'D_prev4', 'D_prev5', 'D_prev6']
['D_prev1_NEW', 'D_prev2_NEW', 'D_prev3_NEW', 'D_prev4_NEW', 'D_prev5_NEW', 'D_prev6_NEW']
['ATM_D_prev1', 'ATM_D_prev2', 'ATM_D_prev3', 'ATM_D_prev4', 'ATM_D_prev5', 'ATM_D_prev6']
['ATM_D_prev1_NEW', 'ATM_D_prev2_NEW', 'ATM_D_prev3_NEW', 'ATM_D_prev4_NEW', 'ATM_D_prev5_NEW', 'ATM_D_prev6_NEW']
['BRANCH_C_prev1', 'BRANCH_C_prev2', 'BRANCH_C_prev3', 'BRANCH_C_prev4', 'BRANCH_C_prev5', 'BRANCH_C_prev6']
['BRANCH_C_prev1_NEW', 'BRANCH_C_prev2_NEW', 'BRANCH_C_prev3_NEW', 'BRANCH_C_prev4_NEW', 'BRANCH_C_prev5_NEW', 'BRANCH_C_prev6_NEW']
['BRANCH_D_prev1', 'BRANCH_D_prev2', 'BRANCH_D_prev3', 'BRANCH_D_prev4', 'BRANCH_D_prev5', 'BRANCH_D_prev6']
['BRANCH_D_prev1_NEW', 'BRANCH_D_prev2_NEW', 'BRANCH_D_prev3_NEW', 'BRANCH_D_prev4_NEW', 'BRANCH_D_prev5_NEW', 'BRANCH_D_prev6_

# This is variable treatment

In [191]:
def set_max_value(df,columns,val,new_cols="NONE"):
    if(new_cols == "NONE"):
        new_cols = columns
    if(type(columns) is list):
        for col,new_col in zip(columns,new_cols):
            df[new_col] = df[col]
            df.loc[df[new_col]>val,new_col] = val
    else:
        df.loc[df[columns]>val,new_cols] = val

In [192]:
def set_min_value(df,columns,val,new_cols="NONE"):
    if(new_cols == "NONE"):
        new_cols = columns
    if(type(columns) is list):
        for col,new_col in zip(columns,new_cols):
            df[new_col] = df[col]
            df.loc[df[new_col]<val,new_col] = val
    else:
        df.loc[df[columns]<val,new_cols] = val

In [193]:
def set_null_value(df,columns,val=0,new_cols="NONE"):
    if(new_cols == "NONE"):
        new_cols = columns
    if(type(columns) is list):
        for col,new_col in zip(columns,new_cols):
            df[new_col] = df[col]
            df.loc[df[new_col].isnull(),new_col] = val
    else:
        df.loc[df[columns].isnull(),new_cols] = val

In [194]:
def get_cols(col_prefix):
    cols = []
    new_cols=[]
    for i in np.array([1,2,3,4,5,6]):
        cols.append(col_prefix + str(i))
        new_cols.append(col_prefix + str(i)+"_NEW")
    return cols,new_cols

In [195]:
def fix_goddamn_hell(DF):
	c
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,3400000,new_cols)


	DF[cols].describe()

	cols,new_cols = get_cols("D_prev")
	print(cols)

	print(new_cols)
	set_max_value(DF,cols,3400000,new_cols)

	cols,new_cols = get_cols("ATM_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,150000,new_cols)

	cols,new_cols = get_cols("BRANCH_C_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,3200000,new_cols)

	cols,new_cols = get_cols("BRANCH_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,2700000,new_cols)

	cols,new_cols = get_cols("IB_C_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,23000,new_cols)

	cols,new_cols = get_cols("IB_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,450000,new_cols)

	cols,new_cols = get_cols("POS_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,45000,new_cols)


	cols,new_cols = get_cols("count_C_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,11,new_cols)

	cols,new_cols = get_cols("count_D_prev")
	print(cols)
	print(new_cols)
	#DF[cols].describe([.1,.2,.5,.7,.8,.9,.95,.99])
	set_max_value(DF,cols,45,new_cols)

	cols,new_cols = get_cols("COUNT_ATM_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,14,new_cols)

	cols,new_cols = get_cols("COUNT_BRANCH_C_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,10,new_cols)

	cols,new_cols = get_cols("COUNT_BRANCH_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,13,new_cols)

	cols,new_cols = get_cols("COUNT_IB_C_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,2,new_cols)

	cols,new_cols = get_cols("COUNT_IB_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,30,new_cols)

	cols,new_cols = get_cols("COUNT_MB_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,3,new_cols)

	cols,new_cols = get_cols("COUNT_POS_D_prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,14,new_cols)

	cols,new_cols = get_cols("custinit_CR_amt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,1200000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("custinit_DR_amt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,3000000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("custinit_CR_cnt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,7,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("custinit_DR_cnt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,42,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("ATM_amt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,145000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("ATM_CW_Amt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,145000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("ATM_CW_Cnt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,14,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	DF["BRN_CW_Amt_prev_ntnull_cnt"] = DF[["BRN_CW_Amt_prev1","BRN_CW_Amt_prev2","BRN_CW_Amt_prev3","BRN_CW_Amt_prev4","BRN_CW_Amt_prev5"]].sum(axis=1)
	DF.loc[DF["BRN_CW_Amt_prev_ntnull_cnt"]>0,["BRN_CW_Amt_prev1","BRN_CW_Amt_prev2","BRN_CW_Amt_prev3","BRN_CW_Amt_prev4","BRN_CW_Amt_prev5","BRN_CW_Amt_prev_ntnull_cnt"]].tail(10)

	DF["BRN_CW_Cnt_prev_ntnull_cnt"] = DF[["BRN_CW_Cnt_prev1","BRN_CW_Cnt_prev2","BRN_CW_Cnt_prev3","BRN_CW_Cnt_prev4","BRN_CW_Cnt_prev5"]].sum(axis=1)
	DF.loc[DF["BRN_CW_Cnt_prev_ntnull_cnt"]>0,["BRN_CW_Cnt_prev1","BRN_CW_Cnt_prev2","BRN_CW_Cnt_prev3","BRN_CW_Cnt_prev4","BRN_CW_Cnt_prev5","BRN_CW_Cnt_prev_ntnull_cnt"]].head()


	cols,new_cols = get_cols("BRN_CASH_Dep_Amt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,300000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("BRN_CASH_Dep_Cnt_prev")
	print(cols)
	print(new_cols)
	set_null_value(DF,cols,0)
	set_max_value(DF,cols,10,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("CNR_prev")
	print(cols)
	print(new_cols)
	set_min_value(DF,cols,-10000,new_cols)
	set_max_value(DF,new_cols,25000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("BAL_prev")
	print(cols)
	print(new_cols)
	set_min_value(DF,cols,-2000,new_cols)
	set_max_value(DF,new_cols,4000000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("EOP_prev")
	print(cols)
	print(new_cols)
	set_min_value(DF,cols,-2000,new_cols)
	set_max_value(DF,new_cols,4000000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	cols,new_cols = get_cols("CR_AMB_Prev")
	print(cols)
	print(new_cols)
	set_max_value(DF,cols,4200000,new_cols)
	DF[new_cols].describe([.2,.5,.7,.9,.95,.99])

	col = "Billpay_Reg_ason_Prev1"
	set_null_value(DF,col)
	DF[col].describe()

	cols=["FD_AMOUNT_BOOK_PrevQ1", "FD_AMOUNT_BOOK_PrevQ2"]
	new_cols=["FD_AMOUNT_BOOK_PrevQ1_NEW", "FD_AMOUNT_BOOK_PrevQ2_NEW"]
	set_max_value(DF,cols,2000000,new_cols)

	cols = ["NO_OF_FD_BOOK_PrevQ1","NO_OF_FD_BOOK_PrevQ2", "NO_OF_RD_BOOK_PrevQ1", "NO_OF_RD_BOOK_PrevQ2"]
	for col in cols:
		set_null_value(DF,col)
	DF[cols].describe()

	col = "dependents"
	set_max_value(DF,col,3)
	set_null_value(DF,col,0.3456)

	non_missing_cat_cols_encoded = encode_cat(DF,non_missing_cat_cols)
	missing_cat_cols_encoded = encode_cat(DF,missing_cat_cols)

	DF["complain_resolved_total"]= DF[cols].sum(axis=1)    
	DF.loc[DF["Req_Resolved_PrevQ1"]>0,["complain_resolved_total","Req_Resolved_PrevQ1","Query_Resolved_PrevQ1","Complaint_Resolved_PrevQ1"]].head()


# Variable selection and model building 

In [396]:
temp_cols_list = ['C_prev1_NEW', 'C_prev2_NEW', 'C_prev3_NEW', 'C_prev4_NEW', 'C_prev5_NEW', 'C_prev6_NEW'] + ['D_prev1_NEW', 'D_prev2_NEW', 'D_prev3_NEW', 'D_prev4_NEW', 'D_prev5_NEW', 'D_prev6_NEW'] + ['ATM_D_prev1_NEW', 'ATM_D_prev2_NEW', 'ATM_D_prev3_NEW', 'ATM_D_prev4_NEW', 'ATM_D_prev5_NEW', 'ATM_D_prev6_NEW']+ ['BRANCH_C_prev1_NEW', 'BRANCH_C_prev2_NEW', 'BRANCH_C_prev3_NEW', 'BRANCH_C_prev4_NEW', 'BRANCH_C_prev5_NEW', 'BRANCH_C_prev6_NEW']+ ['BRANCH_D_prev1_NEW', 'BRANCH_D_prev2_NEW', 'BRANCH_D_prev3_NEW', 'BRANCH_D_prev4_NEW', 'BRANCH_D_prev5_NEW', 'BRANCH_D_prev6_NEW']+ ['IB_C_prev1_NEW', 'IB_C_prev2_NEW', 'IB_C_prev3_NEW', 'IB_C_prev4_NEW', 'IB_C_prev5_NEW', 'IB_C_prev6_NEW']+ ['IB_D_prev1_NEW', 'IB_D_prev2_NEW', 'IB_D_prev3_NEW', 'IB_D_prev4_NEW', 'IB_D_prev5_NEW', 'IB_D_prev6_NEW']+ ['POS_D_prev1_NEW', 'POS_D_prev2_NEW', 'POS_D_prev3_NEW', 'POS_D_prev4_NEW', 'POS_D_prev5_NEW', 'POS_D_prev6_NEW']+ ['count_C_prev1_NEW', 'count_C_prev2_NEW', 'count_C_prev3_NEW', 'count_C_prev4_NEW', 'count_C_prev5_NEW', 'count_C_prev6_NEW']+ ['count_D_prev1_NEW', 'count_D_prev2_NEW', 'count_D_prev3_NEW', 'count_D_prev4_NEW', 'count_D_prev5_NEW', 'count_D_prev6_NEW']+ ['COUNT_ATM_D_prev1_NEW', 'COUNT_ATM_D_prev2_NEW', 'COUNT_ATM_D_prev3_NEW', 'COUNT_ATM_D_prev4_NEW', 'COUNT_ATM_D_prev5_NEW', 'COUNT_ATM_D_prev6_NEW']+ ['COUNT_BRANCH_C_prev1_NEW', 'COUNT_BRANCH_C_prev2_NEW', 'COUNT_BRANCH_C_prev3_NEW', 'COUNT_BRANCH_C_prev4_NEW', 'COUNT_BRANCH_C_prev5_NEW', 'COUNT_BRANCH_C_prev6_NEW']+ ['COUNT_BRANCH_D_prev1_NEW', 'COUNT_BRANCH_D_prev2_NEW', 'COUNT_BRANCH_D_prev3_NEW', 'COUNT_BRANCH_D_prev4_NEW', 'COUNT_BRANCH_D_prev5_NEW', 'COUNT_BRANCH_D_prev6_NEW']+ ['COUNT_IB_C_prev1_NEW', 'COUNT_IB_C_prev2_NEW', 'COUNT_IB_C_prev3_NEW', 'COUNT_IB_C_prev4_NEW', 'COUNT_IB_C_prev5_NEW', 'COUNT_IB_C_prev6_NEW']+ ['COUNT_IB_D_prev1_NEW', 'COUNT_IB_D_prev2_NEW', 'COUNT_IB_D_prev3_NEW', 'COUNT_IB_D_prev4_NEW', 'COUNT_IB_D_prev5_NEW', 'COUNT_IB_D_prev6_NEW']+ ['COUNT_MB_D_prev1_NEW', 'COUNT_MB_D_prev2_NEW', 'COUNT_MB_D_prev3_NEW', 'COUNT_MB_D_prev4_NEW', 'COUNT_MB_D_prev5_NEW', 'COUNT_MB_D_prev6_NEW']+ ['COUNT_POS_D_prev1_NEW', 'COUNT_POS_D_prev2_NEW', 'COUNT_POS_D_prev3_NEW', 'COUNT_POS_D_prev4_NEW', 'COUNT_POS_D_prev5_NEW', 'COUNT_POS_D_prev6_NEW']+ ['custinit_CR_amt_prev1_NEW', 'custinit_CR_amt_prev2_NEW', 'custinit_CR_amt_prev3_NEW', 'custinit_CR_amt_prev4_NEW', 'custinit_CR_amt_prev5_NEW', 'custinit_CR_amt_prev6_NEW']+ ['custinit_DR_amt_prev1_NEW', 'custinit_DR_amt_prev2_NEW', 'custinit_DR_amt_prev3_NEW', 'custinit_DR_amt_prev4_NEW', 'custinit_DR_amt_prev5_NEW', 'custinit_DR_amt_prev6_NEW']+ ['custinit_CR_cnt_prev1_NEW', 'custinit_CR_cnt_prev2_NEW', 'custinit_CR_cnt_prev3_NEW', 'custinit_CR_cnt_prev4_NEW', 'custinit_CR_cnt_prev5_NEW', 'custinit_CR_cnt_prev6_NEW']+ ['custinit_DR_cnt_prev1_NEW', 'custinit_DR_cnt_prev2_NEW', 'custinit_DR_cnt_prev3_NEW', 'custinit_DR_cnt_prev4_NEW', 'custinit_DR_cnt_prev5_NEW', 'custinit_DR_cnt_prev6_NEW']+ ['ATM_amt_prev1_NEW', 'ATM_amt_prev2_NEW', 'ATM_amt_prev3_NEW', 'ATM_amt_prev4_NEW', 'ATM_amt_prev5_NEW', 'ATM_amt_prev6_NEW']+ ['ATM_CW_Amt_prev1_NEW', 'ATM_CW_Amt_prev2_NEW', 'ATM_CW_Amt_prev3_NEW', 'ATM_CW_Amt_prev4_NEW', 'ATM_CW_Amt_prev5_NEW', 'ATM_CW_Amt_prev6_NEW']+ ['ATM_CW_Cnt_prev1_NEW', 'ATM_CW_Cnt_prev2_NEW', 'ATM_CW_Cnt_prev3_NEW', 'ATM_CW_Cnt_prev4_NEW', 'ATM_CW_Cnt_prev5_NEW', 'ATM_CW_Cnt_prev6_NEW']+ ['BRN_CW_Amt_prev_ntnull_cnt']+ ['BRN_CW_Cnt_prev_ntnull_cnt'] + ['BRN_CASH_Dep_Amt_prev1_NEW', 'BRN_CASH_Dep_Amt_prev2_NEW', 'BRN_CASH_Dep_Amt_prev3_NEW', 'BRN_CASH_Dep_Amt_prev4_NEW', 'BRN_CASH_Dep_Amt_prev5_NEW', 'BRN_CASH_Dep_Amt_prev6_NEW']+ ['BRN_CASH_Dep_Cnt_prev1_NEW', 'BRN_CASH_Dep_Cnt_prev2_NEW', 'BRN_CASH_Dep_Cnt_prev3_NEW', 'BRN_CASH_Dep_Cnt_prev4_NEW', 'BRN_CASH_Dep_Cnt_prev5_NEW', 'BRN_CASH_Dep_Cnt_prev6_NEW']+ ['CNR_prev1_NEW', 'CNR_prev2_NEW', 'CNR_prev3_NEW', 'CNR_prev4_NEW', 'CNR_prev5_NEW', 'CNR_prev6_NEW']+ ['BAL_prev1_NEW', 'BAL_prev2_NEW', 'BAL_prev3_NEW', 'BAL_prev4_NEW', 'BAL_prev5_NEW', 'BAL_prev6_NEW']+ ['EOP_prev1_NEW', 'EOP_prev2_NEW', 'EOP_prev3_NEW', 'EOP_prev4_NEW', 'EOP_prev5_NEW', 'EOP_prev6_NEW']+ ['CR_AMB_Prev1_NEW', 'CR_AMB_Prev2_NEW', 'CR_AMB_Prev3_NEW', 'CR_AMB_Prev4_NEW', 'CR_AMB_Prev5_NEW', 'CR_AMB_Prev6_NEW']+ ['Billpay_Reg_ason_Prev1','FD_AMOUNT_BOOK_PrevQ1', 'FD_AMOUNT_BOOK_PrevQ2']+ ['NO_OF_FD_BOOK_PrevQ1','NO_OF_FD_BOOK_PrevQ2', 'NO_OF_RD_BOOK_PrevQ1', 'NO_OF_RD_BOOK_PrevQ2']+ ['vintage','NO_OF_Accs','dependents'] + ['complain_resolved_total']+["INV_MISSING_VALUE_COUNT"]
temp_cols_list = temp_cols_list + non_missing_cat_cols_encoded + missing_cat_cols_encoded 

In [284]:
temp_cols_list = ['C_prev1', 'C_prev2', 'C_prev3', 'C_prev4', 'C_prev5', 'C_prev6'] + ['D_prev1', 'D_prev2', 'D_prev3', 'D_prev4', 'D_prev5', 'D_prev6'] + ['ATM_D_prev1', 'ATM_D_prev2', 'ATM_D_prev3', 'ATM_D_prev4', 'ATM_D_prev5', 'ATM_D_prev6']+ ['BRANCH_C_prev1', 'BRANCH_C_prev2', 'BRANCH_C_prev3', 'BRANCH_C_prev4', 'BRANCH_C_prev5', 'BRANCH_C_prev6']+ ['BRANCH_D_prev1', 'BRANCH_D_prev2', 'BRANCH_D_prev3', 'BRANCH_D_prev4', 'BRANCH_D_prev5', 'BRANCH_D_prev6']+ ['IB_C_prev1', 'IB_C_prev2', 'IB_C_prev3', 'IB_C_prev4', 'IB_C_prev5', 'IB_C_prev6']+ ['IB_D_prev1', 'IB_D_prev2', 'IB_D_prev3', 'IB_D_prev4', 'IB_D_prev5', 'IB_D_prev6']+ ['POS_D_prev1', 'POS_D_prev2', 'POS_D_prev3', 'POS_D_prev4', 'POS_D_prev5', 'POS_D_prev6']+ ['count_C_prev1', 'count_C_prev2', 'count_C_prev3', 'count_C_prev4', 'count_C_prev5', 'count_C_prev6']+ ['count_D_prev1', 'count_D_prev2', 'count_D_prev3', 'count_D_prev4', 'count_D_prev5', 'count_D_prev6']+ ['COUNT_ATM_D_prev1', 'COUNT_ATM_D_prev2', 'COUNT_ATM_D_prev3', 'COUNT_ATM_D_prev4', 'COUNT_ATM_D_prev5', 'COUNT_ATM_D_prev6']+ ['COUNT_BRANCH_C_prev1', 'COUNT_BRANCH_C_prev2', 'COUNT_BRANCH_C_prev3', 'COUNT_BRANCH_C_prev4', 'COUNT_BRANCH_C_prev5', 'COUNT_BRANCH_C_prev6']+ ['COUNT_BRANCH_D_prev1', 'COUNT_BRANCH_D_prev2', 'COUNT_BRANCH_D_prev3', 'COUNT_BRANCH_D_prev4', 'COUNT_BRANCH_D_prev5', 'COUNT_BRANCH_D_prev6']+ ['COUNT_IB_C_prev1', 'COUNT_IB_C_prev2', 'COUNT_IB_C_prev3', 'COUNT_IB_C_prev4', 'COUNT_IB_C_prev5', 'COUNT_IB_C_prev6']+ ['COUNT_IB_D_prev1', 'COUNT_IB_D_prev2', 'COUNT_IB_D_prev3', 'COUNT_IB_D_prev4', 'COUNT_IB_D_prev5', 'COUNT_IB_D_prev6']+ ['COUNT_MB_D_prev1', 'COUNT_MB_D_prev2', 'COUNT_MB_D_prev3', 'COUNT_MB_D_prev4', 'COUNT_MB_D_prev5', 'COUNT_MB_D_prev6']+ ['COUNT_POS_D_prev1', 'COUNT_POS_D_prev2', 'COUNT_POS_D_prev3', 'COUNT_POS_D_prev4', 'COUNT_POS_D_prev5', 'COUNT_POS_D_prev6']+ ['custinit_CR_amt_prev1', 'custinit_CR_amt_prev2', 'custinit_CR_amt_prev3', 'custinit_CR_amt_prev4', 'custinit_CR_amt_prev5', 'custinit_CR_amt_prev6']+ ['custinit_DR_amt_prev1', 'custinit_DR_amt_prev2', 'custinit_DR_amt_prev3', 'custinit_DR_amt_prev4', 'custinit_DR_amt_prev5', 'custinit_DR_amt_prev6']+ ['custinit_CR_cnt_prev1', 'custinit_CR_cnt_prev2', 'custinit_CR_cnt_prev3', 'custinit_CR_cnt_prev4', 'custinit_CR_cnt_prev5', 'custinit_CR_cnt_prev6']+ ['custinit_DR_cnt_prev1', 'custinit_DR_cnt_prev2', 'custinit_DR_cnt_prev3', 'custinit_DR_cnt_prev4', 'custinit_DR_cnt_prev5', 'custinit_DR_cnt_prev6']+ ['ATM_amt_prev1', 'ATM_amt_prev2', 'ATM_amt_prev3', 'ATM_amt_prev4', 'ATM_amt_prev5', 'ATM_amt_prev6']+ ['ATM_CW_Amt_prev1', 'ATM_CW_Amt_prev2', 'ATM_CW_Amt_prev3', 'ATM_CW_Amt_prev4', 'ATM_CW_Amt_prev5', 'ATM_CW_Amt_prev6']+ ['ATM_CW_Cnt_prev1', 'ATM_CW_Cnt_prev2', 'ATM_CW_Cnt_prev3', 'ATM_CW_Cnt_prev4', 'ATM_CW_Cnt_prev5', 'ATM_CW_Cnt_prev6']+ ['BRN_CW_Amt_prev_ntnull_cnt']+ ['BRN_CW_Cnt_prev_ntnull_cnt'] + ['BRN_CASH_Dep_Amt_prev1', 'BRN_CASH_Dep_Amt_prev2', 'BRN_CASH_Dep_Amt_prev3', 'BRN_CASH_Dep_Amt_prev4', 'BRN_CASH_Dep_Amt_prev5', 'BRN_CASH_Dep_Amt_prev6']+ ['BRN_CASH_Dep_Cnt_prev1', 'BRN_CASH_Dep_Cnt_prev2', 'BRN_CASH_Dep_Cnt_prev3', 'BRN_CASH_Dep_Cnt_prev4', 'BRN_CASH_Dep_Cnt_prev5', 'BRN_CASH_Dep_Cnt_prev6']+ ['CNR_prev1', 'CNR_prev2', 'CNR_prev3', 'CNR_prev4', 'CNR_prev5', 'CNR_prev6']+ ['BAL_prev1', 'BAL_prev2', 'BAL_prev3', 'BAL_prev4', 'BAL_prev5', 'BAL_prev6']+ ['EOP_prev1', 'EOP_prev2', 'EOP_prev3', 'EOP_prev4', 'EOP_prev5', 'EOP_prev6']+ ['CR_AMB_Prev1', 'CR_AMB_Prev2', 'CR_AMB_Prev3', 'CR_AMB_Prev4', 'CR_AMB_Prev5', 'CR_AMB_Prev6']+ ['Billpay_Reg_ason_Prev1','FD_AMOUNT_BOOK_PrevQ1', 'FD_AMOUNT_BOOK_PrevQ2']+ ['NO_OF_FD_BOOK_PrevQ1','NO_OF_FD_BOOK_PrevQ2', 'NO_OF_RD_BOOK_PrevQ1', 'NO_OF_RD_BOOK_PrevQ2']+ ['vintage','NO_OF_Accs','dependents'] + ['complain_resolved_total']+["INV_MISSING_VALUE_COUNT"]
temp_cols_list = temp_cols_list + non_missing_cat_cols_encoded + missing_cat_cols_encoded 

In [416]:
train["Responders_scaled"] =  train["Responders"].map({1:10,0:0})

In [413]:
#gbc = GradientBoostingClassifier(learning_rate=0.005,subsample=0.85, random_state=10)
#clf = RandomForestClassifier(n_estimators = 50, n_jobs =-1,class_weight ={1: 10, 0: 1})
# gbc = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1200,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=20,
# warm_start=True)
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
#regr = RandomForestRegressor(n_estimators = 200,n_jobs=-1)


regr = GradientBoostingRegressor(learning_rate=0.005, n_estimators=1200,subsample=0.85, random_state=10, max_features=20, warm_start=True)

In [450]:
temp_cols_list = model_cols_list
gbc = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1200,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=20,
 warm_start=True)
gbc.fit(train[temp_cols_list], train["Responders"], sample_weight = train["Responders"].map({1:10,0:1}))
predict_and_save_file(gbc,test,temp_cols_list)

C:\Users\Pavan\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [415]:
predict_and_save_file(regr,test,temp_cols_list,prob=False)

C:\Users\Pavan\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
x = train["brn_code"].value_counts()
y = train.loc[train["Responders"] ==1,"brn_code"].value_counts()

branches = x.to_frame()

branches["code"]=x.index

branches_churn = y.to_frame()
branches_churn["code"]=y.index

branches = branches.merge(branches_churn, on='code', how='left')
branches = branches.fillna(0)
branches.columns = ["customers","brn_code","churners"]
branches["churn_percentage"] = branches["churners"]/branches["customers"]
branches["customer_percentage"] = 100*branches["customers"]/sum(branches["customers"])


train = train.merge(branches,on="brn_code",how="left")
train["customer_percentage"] = train["customer_percentage"].fillna(0)
train["churn_percentage"] = train["churn_percentage"].fillna(0)


test = test.merge(branches,on="brn_code",how="left")
test["customer_percentage"] = test["customer_percentage"].fillna(0)
test["churn_percentage"] = test["churn_percentage"].fillna(0)

temp_cols_list.append("customer_percentage")
temp_cols_list.append("churn_percentage")

##put gbc fit

gbc.fit(train[temp_cols_list], train["Responders"], sample_weight = train["Responders"].map({1:10,0:1}))
predict_and_save_file(gbc,test,temp_cols_list)



#predict and save



In [307]:
train.head()

,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,...,Billpay_Reg_ason_Prev1_NEN,Charges_cnt_PrevQ1_NEN,FRX_PrevQ1_NEN,RBI_Class_AuditEN,gender_binEN,EMAIL_UNSUBSCRIBEEN_MIS

In [460]:
def cross_validation(mod,cols,target="Responders",class_weights=False):
    skf = StratifiedKFold(n_splits=3)
    for train_index, test_index in skf.split(train[cols], train[target]):
        #print str(train_index.shape) + " " + str(test_index.shape)
        if(class_weights):
            mod.fit(train.loc[train_index,cols], train.loc[train_index,target],sample_weight = train.loc[train_index,target].map({1:10,0:1}))
        else:
            mod.fit(train.loc[train_index,cols], train.loc[train_index,target])
        train.loc[test_index,"predicted"] = mod.predict_proba(train.loc[test_index,cols])[:,1]
        calculate_lift(train.loc[test_index,["UCIC_ID","Responders","predicted"]])
    feature_importance(mod,cols)

In [412]:
feature_importance(gbc,temp_cols_list,True)

0.0826832439762 EOP_prev_CV_1 
0.0715599356916 churn_percentage 
0.0636740834486 EOP_prev1_NEW 
0.0450981359624 CR_AMB_Drop_Build_1 
0.0334779132864 EOP_prev_CV 
0.0258272648114 BAL_prev_CV 
0.0255705668855 CR_AMB_Prev_CV 
0.0211508991601 BAL_prev_CV_1 
0.018917254901 CR_AMB_Prev_CV_1 
0.0187418133907 BAL_prev1_NEW 
0.0186728747525 FINAL_WORTH_prev1EN 
0.0183150164464 CR_AMB_Prev1_NEW 
0.0151589959158 ENGAGEMENT_TAG_prev1EN 
0.0148627259791 CR_AMB_Drop_Build_3 
0.014825515014 D_prev1_NEW 
0.0146748958061 I_AQB_PrevQ1 
0.0143902589908 I_CR_AQB_PrevQ1 
0.0132118776617 vintage 
0.0115319586751 custinit_DR_amt_prev1_NEW 
0.0114783706289 BAL_prev3_NEW 
0.0110765267042 CNR_prev_CV 
0.0104339187737 CR_AMB_Prev2_NEW 
0.0100937386407 CR_AMB_Prev3_NEW 
0.0094489286889 BAL_prev2_NEW 
0.00935869259523 CR_AMB_Drop_Build_2 
0.0092370913799 EOP_prev2_NEW 
0.00884940518597 BAL_prev6_NEW 
0.00847639294384 age 
0.00834385658439 CR_AMB_Prev6_NEW 
0.00822531215449 EOP_prev6_NEW 
0.00822496764336 D_prev_CV

In [165]:
temp_corr_cols_list = ['C_prev1', 'C_prev2', 'C_prev3', 'C_prev4', 'C_prev5', 'C_prev6'] + ['D_prev1', 'D_prev2', 'D_prev3', 'D_prev4', 'D_prev5', 'D_prev6'] + ['ATM_D_prev1', 'ATM_D_prev2', 'ATM_D_prev3', 'ATM_D_prev4', 'ATM_D_prev5', 'ATM_D_prev6']
corr_matrix = train[temp_corr_cols_list].corr()
high_corr_var=np.where(corr_matrix>0.7)
high_corr_var=[(corr_matrix.index[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
print(high_corr_var)


[('C_prev1', 'D_prev1'), ('C_prev2', 'D_prev2'), ('C_prev3', 'D_prev3'), ('C_prev4', 'D_prev4'), ('C_prev5', 'D_prev5'), ('C_prev6', 'D_prev6'), ('D_prev1', 'D_prev2')]


In [ ]:
useless_cols = []
feature_importance(regr,temp_cols_list,True)

In [222]:
# test = pd.read_csv("C:\\Users\\Pavan\\Desktop\\Avidya\\churn_prediction\\test.csv")

# add_missing_val_count(test)
# add_missing_val_cat_cols(test,missing_cat_cols)
# encode_cat(test,non_missing_cat_cols)
# encode_cat(test,missing_cat_cols,suffix="EN_MIS")
# treat_complaints(test)

# fix_goddamn_hell(test)


C:\Users\Pavan\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [231]:
useless_cols

['FRX_PrevQ1_NEN',
 'TL_TAG_LIVEEN',
 'AGRI_TAG_LIVEEN',
 'EDU_TAG_LIVEEN',
 'LAS_TAG_LIVEEN']

# FINDING CORRELATION AND REDUCING COLUMNS

In [238]:
temp_corr_cols_list = temp_cols_list
corr_matrix = train[temp_corr_cols_list].corr()
high_corr_var=np.where(abs(corr_matrix)>0.7)
high_corr_var=[(corr_matrix.index[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
print(high_corr_var)


[('C_prev1_NEW', 'BRANCH_C_prev1_NEW'), ('C_prev2_NEW', 'BRANCH_C_prev2_NEW'), ('C_prev3_NEW', 'BRANCH_C_prev3_NEW'), ('C_prev4_NEW', 'BRANCH_C_prev4_NEW'), ('C_prev5_NEW', 'BRANCH_C_prev5_NEW'), ('C_prev6_NEW', 'BRANCH_C_prev6_NEW'), ('D_prev1_NEW', 'BRANCH_D_prev1_NEW'), ('D_prev1_NEW', 'custinit_DR_amt_prev1_NEW'), ('D_prev2_NEW', 'BRANCH_D_prev2_NEW'), ('D_prev2_NEW', 'custinit_DR_amt_prev2_NEW'), ('D_prev3_NEW', 'BRANCH_D_prev3_NEW'), ('D_prev3_NEW', 'custinit_DR_amt_prev3_NEW'), ('D_prev4_NEW', 'BRANCH_D_prev4_NEW'), ('D_prev4_NEW', 'custinit_DR_amt_prev4_NEW'), ('D_prev5_NEW', 'BRANCH_D_prev5_NEW'), ('D_prev5_NEW', 'custinit_DR_amt_prev5_NEW'), ('D_prev6_NEW', 'BRANCH_D_prev6_NEW'), ('D_prev6_NEW', 'custinit_DR_amt_prev6_NEW'), ('ATM_D_prev1_NEW', 'COUNT_ATM_D_prev1_NEW'), ('ATM_D_prev1_NEW', 'ATM_amt_prev1_NEW'), ('ATM_D_prev1_NEW', 'ATM_CW_Amt_prev1_NEW'), ('ATM_D_prev1_NEW', 'ATM_CW_Cnt_prev1_NEW'), ('ATM_D_prev2_NEW', 'COUNT_ATM_D_prev2_NEW'), ('ATM_D_prev2_NEW', 'ATM_amt_pr

In [262]:
from sklearn.decomposition import PCA
reduc_cols = ['C_prev1_NEW', 'C_prev2_NEW', 'C_prev3_NEW', 'C_prev4_NEW', 'C_prev5_NEW', 'C_prev6_NEW']  + ['BRANCH_C_prev1_NEW', 'BRANCH_C_prev2_NEW', 'BRANCH_C_prev3_NEW', 'BRANCH_C_prev4_NEW', 'BRANCH_C_prev5_NEW', 'BRANCH_C_prev6_NEW']
pca = PCA(n_components = 6)
pca.fit(train[reduc_cols])
X_new = pca.transform(train[reduc_cols])

In [269]:
X_new.shape

(300000, 6)

In [268]:
sum(pca.explained_variance_ratio_)

0.99268241054355311

In [267]:
pca.noise_variance_

2796272694.6569824

In [274]:
non_missing_cont_cols.remove("UCIC_ID")
non_missing_cont_cols.remove("Responders")

In [ ]:
clf = RandomForestClassifier(n_estimators = 50, n_jobs =-1,class_weight ={1: 5, 0: 1})
cross_validation(clf,non_missing_cont_cols,target="Responders")

In [288]:
train["age"].describe()

count             300,000.00
mean                   48.35
std                    17.91
min                     1.00
25%                    36.00
50%                    46.00
75%                    60.00
max                    90.00
Name: age, dtype: float64

In [289]:
train.groupby("Responders")["age"].describe()

Responders       
0           count             248,303.00
            mean                   48.55
            std                    18.04
            min                     1.00
            25%                    36.00
            50%                    46.00
            75%                    61.00
            max                    90.00
1           count              51,697.00
            mean                   47.40
            std                    17.28
            min                     1.00
            25%                    35.00
            50%                    45.00
            75%                    59.00
            max                    90.00
Name: age, dtype: float64

In [292]:
seen = set()
uniq = [x for x in temp_cols_list if x not in seen and not seen.add(x)]  

In [322]:
len(temp_cols_list)

260

In [398]:
temp_cols_list_NEW = ['C_prev1_NEW', 'C_prev2_NEW', 'C_prev3_NEW', 'C_prev4_NEW', 'C_prev5_NEW', 'C_prev6_NEW'] + ['D_prev1_NEW', 'D_prev2_NEW', 'D_prev3_NEW', 'D_prev4_NEW', 'D_prev5_NEW', 'D_prev6_NEW'] + ['ATM_D_prev1_NEW', 'ATM_D_prev2_NEW', 'ATM_D_prev3_NEW', 'ATM_D_prev4_NEW', 'ATM_D_prev5_NEW', 'ATM_D_prev6_NEW']+ ['BRANCH_C_prev1_NEW', 'BRANCH_C_prev2_NEW', 'BRANCH_C_prev3_NEW', 'BRANCH_C_prev4_NEW', 'BRANCH_C_prev5_NEW', 'BRANCH_C_prev6_NEW']+ ['BRANCH_D_prev1_NEW', 'BRANCH_D_prev2_NEW', 'BRANCH_D_prev3_NEW', 'BRANCH_D_prev4_NEW', 'BRANCH_D_prev5_NEW', 'BRANCH_D_prev6_NEW']+ ['IB_C_prev1_NEW', 'IB_C_prev2_NEW', 'IB_C_prev3_NEW', 'IB_C_prev4_NEW', 'IB_C_prev5_NEW', 'IB_C_prev6_NEW']+ ['IB_D_prev1_NEW', 'IB_D_prev2_NEW', 'IB_D_prev3_NEW', 'IB_D_prev4_NEW', 'IB_D_prev5_NEW', 'IB_D_prev6_NEW']+ ['POS_D_prev1_NEW', 'POS_D_prev2_NEW', 'POS_D_prev3_NEW', 'POS_D_prev4_NEW', 'POS_D_prev5_NEW', 'POS_D_prev6_NEW']+ ['count_C_prev1_NEW', 'count_C_prev2_NEW', 'count_C_prev3_NEW', 'count_C_prev4_NEW', 'count_C_prev5_NEW', 'count_C_prev6_NEW']+ ['count_D_prev1_NEW', 'count_D_prev2_NEW', 'count_D_prev3_NEW', 'count_D_prev4_NEW', 'count_D_prev5_NEW', 'count_D_prev6_NEW']+ ['COUNT_ATM_D_prev1_NEW', 'COUNT_ATM_D_prev2_NEW', 'COUNT_ATM_D_prev3_NEW', 'COUNT_ATM_D_prev4_NEW', 'COUNT_ATM_D_prev5_NEW', 'COUNT_ATM_D_prev6_NEW']+ ['COUNT_BRANCH_C_prev1_NEW', 'COUNT_BRANCH_C_prev2_NEW', 'COUNT_BRANCH_C_prev3_NEW', 'COUNT_BRANCH_C_prev4_NEW', 'COUNT_BRANCH_C_prev5_NEW', 'COUNT_BRANCH_C_prev6_NEW']+ ['COUNT_BRANCH_D_prev1_NEW', 'COUNT_BRANCH_D_prev2_NEW', 'COUNT_BRANCH_D_prev3_NEW', 'COUNT_BRANCH_D_prev4_NEW', 'COUNT_BRANCH_D_prev5_NEW', 'COUNT_BRANCH_D_prev6_NEW']+ ['COUNT_IB_C_prev1_NEW', 'COUNT_IB_C_prev2_NEW', 'COUNT_IB_C_prev3_NEW', 'COUNT_IB_C_prev4_NEW', 'COUNT_IB_C_prev5_NEW', 'COUNT_IB_C_prev6_NEW']+ ['COUNT_IB_D_prev1_NEW', 'COUNT_IB_D_prev2_NEW', 'COUNT_IB_D_prev3_NEW', 'COUNT_IB_D_prev4_NEW', 'COUNT_IB_D_prev5_NEW', 'COUNT_IB_D_prev6_NEW']+ ['COUNT_MB_D_prev1_NEW', 'COUNT_MB_D_prev2_NEW', 'COUNT_MB_D_prev3_NEW', 'COUNT_MB_D_prev4_NEW', 'COUNT_MB_D_prev5_NEW', 'COUNT_MB_D_prev6_NEW']+ ['COUNT_POS_D_prev1_NEW', 'COUNT_POS_D_prev2_NEW', 'COUNT_POS_D_prev3_NEW', 'COUNT_POS_D_prev4_NEW', 'COUNT_POS_D_prev5_NEW', 'COUNT_POS_D_prev6_NEW']+ ['custinit_CR_amt_prev1_NEW', 'custinit_CR_amt_prev2_NEW', 'custinit_CR_amt_prev3_NEW', 'custinit_CR_amt_prev4_NEW', 'custinit_CR_amt_prev5_NEW', 'custinit_CR_amt_prev6_NEW']+ ['custinit_DR_amt_prev1_NEW', 'custinit_DR_amt_prev2_NEW', 'custinit_DR_amt_prev3_NEW', 'custinit_DR_amt_prev4_NEW', 'custinit_DR_amt_prev5_NEW', 'custinit_DR_amt_prev6_NEW']+ ['custinit_CR_cnt_prev1_NEW', 'custinit_CR_cnt_prev2_NEW', 'custinit_CR_cnt_prev3_NEW', 'custinit_CR_cnt_prev4_NEW', 'custinit_CR_cnt_prev5_NEW', 'custinit_CR_cnt_prev6_NEW']+ ['custinit_DR_cnt_prev1_NEW', 'custinit_DR_cnt_prev2_NEW', 'custinit_DR_cnt_prev3_NEW', 'custinit_DR_cnt_prev4_NEW', 'custinit_DR_cnt_prev5_NEW', 'custinit_DR_cnt_prev6_NEW']+ ['ATM_amt_prev1_NEW', 'ATM_amt_prev2_NEW', 'ATM_amt_prev3_NEW', 'ATM_amt_prev4_NEW', 'ATM_amt_prev5_NEW', 'ATM_amt_prev6_NEW']+ ['ATM_CW_Amt_prev1_NEW', 'ATM_CW_Amt_prev2_NEW', 'ATM_CW_Amt_prev3_NEW', 'ATM_CW_Amt_prev4_NEW', 'ATM_CW_Amt_prev5_NEW', 'ATM_CW_Amt_prev6_NEW']+ ['ATM_CW_Cnt_prev1_NEW', 'ATM_CW_Cnt_prev2_NEW', 'ATM_CW_Cnt_prev3_NEW', 'ATM_CW_Cnt_prev4_NEW', 'ATM_CW_Cnt_prev5_NEW', 'ATM_CW_Cnt_prev6_NEW']+ ['BRN_CW_Amt_prev_ntnull_cnt']+ ['BRN_CW_Cnt_prev_ntnull_cnt'] + ['BRN_CASH_Dep_Amt_prev1_NEW', 'BRN_CASH_Dep_Amt_prev2_NEW', 'BRN_CASH_Dep_Amt_prev3_NEW', 'BRN_CASH_Dep_Amt_prev4_NEW', 'BRN_CASH_Dep_Amt_prev5_NEW', 'BRN_CASH_Dep_Amt_prev6_NEW']+ ['BRN_CASH_Dep_Cnt_prev1_NEW', 'BRN_CASH_Dep_Cnt_prev2_NEW', 'BRN_CASH_Dep_Cnt_prev3_NEW', 'BRN_CASH_Dep_Cnt_prev4_NEW', 'BRN_CASH_Dep_Cnt_prev5_NEW', 'BRN_CASH_Dep_Cnt_prev6_NEW']+ ['CNR_prev1_NEW', 'CNR_prev2_NEW', 'CNR_prev3_NEW', 'CNR_prev4_NEW', 'CNR_prev5_NEW', 'CNR_prev6_NEW']+ ['BAL_prev1_NEW', 'BAL_prev2_NEW', 'BAL_prev3_NEW', 'BAL_prev4_NEW', 'BAL_prev5_NEW', 'BAL_prev6_NEW']+ ['EOP_prev1_NEW', 'EOP_prev2_NEW', 'EOP_prev3_NEW', 'EOP_prev4_NEW', 'EOP_prev5_NEW', 'EOP_prev6_NEW']+ ['CR_AMB_Prev1_NEW', 'CR_AMB_Prev2_NEW', 'CR_AMB_Prev3_NEW', 'CR_AMB_Prev4_NEW', 'CR_AMB_Prev5_NEW', 'CR_AMB_Prev6_NEW']+ ['Billpay_Reg_ason_Prev1','FD_AMOUNT_BOOK_PrevQ1', 'FD_AMOUNT_BOOK_PrevQ2']+ ['NO_OF_FD_BOOK_PrevQ1','NO_OF_FD_BOOK_PrevQ2', 'NO_OF_RD_BOOK_PrevQ1', 'NO_OF_RD_BOOK_PrevQ2']+ ['vintage','NO_OF_Accs','dependents'] + ['complain_resolved_total']+["INV_MISSING_VALUE_COUNT"]
temp_cols_list = ['C_prev1', 'C_prev2', 'C_prev3', 'C_prev4', 'C_prev5', 'C_prev6'] + ['D_prev1', 'D_prev2', 'D_prev3', 'D_prev4', 'D_prev5', 'D_prev6'] + ['ATM_D_prev1', 'ATM_D_prev2', 'ATM_D_prev3', 'ATM_D_prev4', 'ATM_D_prev5', 'ATM_D_prev6']+ ['BRANCH_C_prev1', 'BRANCH_C_prev2', 'BRANCH_C_prev3', 'BRANCH_C_prev4', 'BRANCH_C_prev5', 'BRANCH_C_prev6']+ ['BRANCH_D_prev1', 'BRANCH_D_prev2', 'BRANCH_D_prev3', 'BRANCH_D_prev4', 'BRANCH_D_prev5', 'BRANCH_D_prev6']+ ['IB_C_prev1', 'IB_C_prev2', 'IB_C_prev3', 'IB_C_prev4', 'IB_C_prev5', 'IB_C_prev6']+ ['IB_D_prev1', 'IB_D_prev2', 'IB_D_prev3', 'IB_D_prev4', 'IB_D_prev5', 'IB_D_prev6']+ ['POS_D_prev1', 'POS_D_prev2', 'POS_D_prev3', 'POS_D_prev4', 'POS_D_prev5', 'POS_D_prev6']+ ['count_C_prev1', 'count_C_prev2', 'count_C_prev3', 'count_C_prev4', 'count_C_prev5', 'count_C_prev6']+ ['count_D_prev1', 'count_D_prev2', 'count_D_prev3', 'count_D_prev4', 'count_D_prev5', 'count_D_prev6']+ ['COUNT_ATM_D_prev1', 'COUNT_ATM_D_prev2', 'COUNT_ATM_D_prev3', 'COUNT_ATM_D_prev4', 'COUNT_ATM_D_prev5', 'COUNT_ATM_D_prev6']+ ['COUNT_BRANCH_C_prev1', 'COUNT_BRANCH_C_prev2', 'COUNT_BRANCH_C_prev3', 'COUNT_BRANCH_C_prev4', 'COUNT_BRANCH_C_prev5', 'COUNT_BRANCH_C_prev6']+ ['COUNT_BRANCH_D_prev1', 'COUNT_BRANCH_D_prev2', 'COUNT_BRANCH_D_prev3', 'COUNT_BRANCH_D_prev4', 'COUNT_BRANCH_D_prev5', 'COUNT_BRANCH_D_prev6']+ ['COUNT_IB_C_prev1', 'COUNT_IB_C_prev2', 'COUNT_IB_C_prev3', 'COUNT_IB_C_prev4', 'COUNT_IB_C_prev5', 'COUNT_IB_C_prev6']+ ['COUNT_IB_D_prev1', 'COUNT_IB_D_prev2', 'COUNT_IB_D_prev3', 'COUNT_IB_D_prev4', 'COUNT_IB_D_prev5', 'COUNT_IB_D_prev6']+ ['COUNT_MB_D_prev1', 'COUNT_MB_D_prev2', 'COUNT_MB_D_prev3', 'COUNT_MB_D_prev4', 'COUNT_MB_D_prev5', 'COUNT_MB_D_prev6']+ ['COUNT_POS_D_prev1', 'COUNT_POS_D_prev2', 'COUNT_POS_D_prev3', 'COUNT_POS_D_prev4', 'COUNT_POS_D_prev5', 'COUNT_POS_D_prev6']+ ['custinit_CR_amt_prev1', 'custinit_CR_amt_prev2', 'custinit_CR_amt_prev3', 'custinit_CR_amt_prev4', 'custinit_CR_amt_prev5', 'custinit_CR_amt_prev6']+ ['custinit_DR_amt_prev1', 'custinit_DR_amt_prev2', 'custinit_DR_amt_prev3', 'custinit_DR_amt_prev4', 'custinit_DR_amt_prev5', 'custinit_DR_amt_prev6']+ ['custinit_CR_cnt_prev1', 'custinit_CR_cnt_prev2', 'custinit_CR_cnt_prev3', 'custinit_CR_cnt_prev4', 'custinit_CR_cnt_prev5', 'custinit_CR_cnt_prev6']+ ['custinit_DR_cnt_prev1', 'custinit_DR_cnt_prev2', 'custinit_DR_cnt_prev3', 'custinit_DR_cnt_prev4', 'custinit_DR_cnt_prev5', 'custinit_DR_cnt_prev6']+ ['ATM_amt_prev1', 'ATM_amt_prev2', 'ATM_amt_prev3', 'ATM_amt_prev4', 'ATM_amt_prev5', 'ATM_amt_prev6']+ ['ATM_CW_Amt_prev1', 'ATM_CW_Amt_prev2', 'ATM_CW_Amt_prev3', 'ATM_CW_Amt_prev4', 'ATM_CW_Amt_prev5', 'ATM_CW_Amt_prev6']+ ['ATM_CW_Cnt_prev1', 'ATM_CW_Cnt_prev2', 'ATM_CW_Cnt_prev3', 'ATM_CW_Cnt_prev4', 'ATM_CW_Cnt_prev5', 'ATM_CW_Cnt_prev6']+ ['BRN_CW_Amt_prev_ntnull_cnt']+ ['BRN_CW_Cnt_prev_ntnull_cnt'] + ['BRN_CASH_Dep_Amt_prev1', 'BRN_CASH_Dep_Amt_prev2', 'BRN_CASH_Dep_Amt_prev3', 'BRN_CASH_Dep_Amt_prev4', 'BRN_CASH_Dep_Amt_prev5', 'BRN_CASH_Dep_Amt_prev6']+ ['BRN_CASH_Dep_Cnt_prev1', 'BRN_CASH_Dep_Cnt_prev2', 'BRN_CASH_Dep_Cnt_prev3', 'BRN_CASH_Dep_Cnt_prev4', 'BRN_CASH_Dep_Cnt_prev5', 'BRN_CASH_Dep_Cnt_prev6']+ ['CNR_prev1', 'CNR_prev2', 'CNR_prev3', 'CNR_prev4', 'CNR_prev5', 'CNR_prev6']+ ['BAL_prev1', 'BAL_prev2', 'BAL_prev3', 'BAL_prev4', 'BAL_prev5', 'BAL_prev6']+ ['EOP_prev1', 'EOP_prev2', 'EOP_prev3', 'EOP_prev4', 'EOP_prev5', 'EOP_prev6']+ ['CR_AMB_Prev1', 'CR_AMB_Prev2', 'CR_AMB_Prev3', 'CR_AMB_Prev4', 'CR_AMB_Prev5', 'CR_AMB_Prev6']+ ['Billpay_Reg_ason_Prev1','FD_AMOUNT_BOOK_PrevQ1', 'FD_AMOUNT_BOOK_PrevQ2']+ ['NO_OF_FD_BOOK_PrevQ1','NO_OF_FD_BOOK_PrevQ2', 'NO_OF_RD_BOOK_PrevQ1', 'NO_OF_RD_BOOK_PrevQ2']+ ['vintage','NO_OF_Accs','dependents'] + ['complain_resolved_total']+["INV_MISSING_VALUE_COUNT"]


In [399]:
unused_cols = []
for coli in non_missing_cont_cols:
    
    found = 0
    for colj in temp_cols_list:
        
        if colj==coli:
            found = 1
            break
    if found == 0:
        temp_cols_list_NEW.append(coli)
        temp_cols_list.append(coli)
        unused_cols.append(coli)
temp_cols_list_NEW.remove('brn_code')
temp_cols_list.remove('brn_code')
print(unused_cols)
print("\n")
# unused_cols = []

# for coli in missing_cont_cols:
    
#     found = 0
#     for colj in temp_cols_list:
        
#         if colj==coli:
#             found = 1
#             break
#     if found == 0:
#         temp_cols_list_NEW.append(coli)
#         temp_cols_list.append(coli)
#         unused_cols.append(coli)
# print(unused_cols)

['ATM_C_prev1', 'MB_C_prev1', 'MB_D_prev1', 'POS_C_prev1', 'COUNT_ATM_C_prev1', 'COUNT_MB_C_prev1', 'COUNT_POS_C_prev1', 'ATM_C_prev2', 'MB_C_prev2', 'MB_D_prev2', 'POS_C_prev2', 'COUNT_ATM_C_prev2', 'COUNT_MB_C_prev2', 'COUNT_POS_C_prev2', 'ATM_C_prev3', 'MB_C_prev3', 'MB_D_prev3', 'POS_C_prev3', 'COUNT_ATM_C_prev3', 'COUNT_MB_C_prev3', 'COUNT_POS_C_prev3', 'ATM_C_prev4', 'MB_C_prev4', 'MB_D_prev4', 'POS_C_prev4', 'COUNT_ATM_C_prev4', 'COUNT_MB_C_prev4', 'COUNT_POS_C_prev4', 'ATM_C_prev5', 'MB_C_prev5', 'MB_D_prev5', 'POS_C_prev5', 'COUNT_ATM_C_prev5', 'COUNT_MB_C_prev5', 'COUNT_POS_C_prev5', 'ATM_C_prev6', 'MB_C_prev6', 'MB_D_prev6', 'POS_C_prev6', 'COUNT_ATM_C_prev6', 'COUNT_MB_C_prev6', 'COUNT_POS_C_prev6', 'RD_AMOUNT_BOOK_PrevQ1', 'RD_AMOUNT_BOOK_PrevQ2', 'Total_Invest_in_MF_PrevQ1', 'Total_Invest_in_MF_PrevQ2', 'Dmat_Investing_PrevQ1', 'Dmat_Investing_PrevQ2', 'Charges_PrevQ1', 'CASH_WD_AMT_Last6', 'CASH_WD_CNT_Last6', 'brn_code', 'age', 'I_AQB_PrevQ1', 'I_AQB_PrevQ2', 'I_CR_AQB_

In [402]:
model_cols_list = temp_cols_list_NEW + non_missing_cat_cols_encoded + missing_cat_cols_encoded 

In [354]:
from sklearn.decomposition import PCA
pca = PCA(n_components=80)
new_vars = pca.fit_transform(train[temp_cols_list_NEW], y=None)

In [355]:
sum(pca.explained_variance_ratio_)

0.99999993538120391

In [329]:
new_vars.dtype

dtype('float64')

In [392]:
model_cols_list = []
col ="PCA_"
for i in range(80):
    #train[col+str(i+1)] = new_vars[:,i]
    #train.drop(col+str(i+1),axis=1,inplace=True)
    model_cols_list.append(col+str(i+1))

In [404]:
#model_cols_list = model_cols_list + non_missing_cat_cols_encoded + missing_cat_cols_encoded 
model_cols_list.append("customer_percentage")
model_cols_list.append("churn_percentage")

In [469]:
#from sklearn.linear_model import LogisticRegression
#mod = LogisticRegression(solver="saga",n_jobs=-1,random_state=10,class_weight ={1: 10, 0: 1})

mod = GradientBoostingClassifier(learning_rate=0.005, n_estimators=200,max_depth=9, min_samples_split=100, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=20,
 warm_start=True)
#gbc.fit(train[temp_cols_list], train["Responders"], sample_weight = train["Responders"].map({1:10,0:1}))

In [471]:
#mod = RandomForestClassifier(n_estimators = 100, n_jobs =-1,random_state=10,class_weight ={1: 10, 0: 1},oob_score=True)
#mod = RandomForestClassifier(n_estimators = 200, n_jobs =-1,random_state=10,class_weight ="balanced",oob_score=True)
#the above had more accuracy (n_estimatorts = 100)

cross_validation(mod,model_cols_list,target="Responders",class_weights=True)

C:\Users\Pavan\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\Pavan\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


total_20 20000.2
top_20_responders 11456
tot_responders 17233
LIFT----- 0.6647710787442697  -------
total_20 20000.0
top_20_responders 11808
tot_responders 17232
LIFT----- 0.6852367688022284  -------
total_20 19999.800000000003
top_20_responders 11703
tot_responders 17232
LIFT----- 0.6791434540389972  -------
0.152681574646 EOP_prev_CV_1 
0.104461387735 EOP_prev1_NEW 
0.0835095906969 CR_AMB_Drop_Build_1 
0.0478410606427 CR_AMB_Prev_CV_1 
0.0411798592345 BAL_prev_CV_1 
0.0380918864213 EOP_prev_CV 
0.0292785164887 CR_AMB_Prev1_NEW 
0.0285634114653 BAL_prev_CV 
0.0276610051128 BAL_prev1_NEW 
0.0269408818387 CR_AMB_Prev_CV 
0.0253008895413 D_prev1_NEW 
0.0150945507835 BRANCH_D_prev1_NEW 
0.014829481848 CR_AMB_Drop_Build_3 
0.0145690517414 custinit_DR_amt_prev1_NEW 
0.0132781869253 CNR_prev_CV_1 
0.0124140330255 D_prev_CV_1 
0.0119765746303 FINAL_WORTH_prev1EN 
0.0112691997265 churn_percentage 
0.0111385807369 CR_AMB_Drop_Build_2 
0.0111028326931 BAL_prev3_NEW 
0.0106861517308 CR_AMB_Prev3_

In [468]:
mod

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005, loss='deviance', max_depth=9,
              max_features=20, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=60, min_samples_split=100,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=10, subsample=0.85, verbose=0,
              warm_start=True)

In [ ]:
# from sklearn.ensemble import VotingClassifier
gbc = GradientBoostingClassifier(learning_rate=0.005,random_state=8, n_estimators=400,max_depth=9, min_samples_split=100, min_samples_leaf=60, subsample=0.85,  max_features=20,
 warm_start=True)
# clf = RandomForestClassifier(n_estimators = 400, n_jobs =-1,random_state=8,class_weight ="balanced",oob_score=True)
# clf.fit(train[model_cols_list],train["Responders"])
# votingClassifier = VotingClassifier(estimators=[('rf',clf),('gbc',gbc)],voting="soft")
# votingClassifier.fit(train[model_cols_list],train["Responders"])
gbc.fit(train[temp_cols_list], train["Responders"], sample_weight = train["Responders"].map({1:100,0:10}))
predict_and_save_file(gbc,test,model_cols_list)
                     

In [484]:
# from sklearn.ensemble import ExtraTreesClassifier
# extraTree = ExtraTreesClassifier(n_estimators = 400, n_jobs =-1,random_state=7,class_weight ={1: 100, 0: 1},oob_score=False)
# gbc = GradientBoostingClassifier(learning_rate=0.005,random_state=7, n_estimators=300,max_depth=9, min_samples_split=100, min_samples_leaf=60, subsample=0.85,  max_features=20,
#  warm_start=True)
# clf = RandomForestClassifier(n_estimators = 400, n_jobs =-1,random_state=7,class_weight ="balanced",oob_score=True)
# votingClassifier = VotingClassifier(estimators=[('rf',clf),('gbc',gbc),('ext',extraTree)],voting="hard")
# votingClassifier.fit(train[model_cols_list],train["Responders"])
predict_and_save_file(votingClassifier,test,model_cols_list,prob=False)

C:\Users\Pavan\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
extraTree = ExtraTreesClassifier(n_estimators = 400, n_jobs =-1,random_state=7,class_weight ={1: 10, 0: 1},oob_score=True)
gbc = GradientBoostingClassifier(learning_rate=0.005,random_state=6, n_estimators=800,max_depth=9, min_samples_split=100, min_samples_leaf=60, subsample=0.85,  max_features=20,
 warm_start=True)
clf = RandomForestClassifier(n_estimators = 400, n_jobs =-1,random_state=6,class_weight ="balanced",oob_score=True)
votingClassifier = VotingClassifier(estimators=[('rf',clf),('gbc',gbc),('ext',extraTree)],voting="soft",weights=hts=[3,2,1])
votingClassifier.fit(train[model_cols_list],train["Responders"])
predict_and_save_file(votingClassifier,test,model_cols_list)

In [408]:

def add_cv(col,df):
    cols,new_cols = get_cols(col)
    df[col+"_CV"]=(df[cols].std(axis = 1))/(df[cols].mean(axis = 1))
    df[col+"_CV_1"]=df[cols[0]]/df[cols].sum(axis = 1)
    df[col+"_CV_1"].fillna(0,inplace=True)
    #model_cols_list.append(col+"_CV_1")
    cols.append(col+"_CV_1")
    cols.append(col+"_CV")
    #model_cols_list.append(col+"_CV")
    df[col+"_CV"].fillna(0,inplace=True)
    print(df[cols].head())

In [ ]:
cols = ["D_prev","C_prev","CNR_prev","custinit_DR_amt_prev","BRANCH_D_prev","BAL_prev","EOP_prev","CR_AMB_Prev"]
for col in cols:
    add_cv(col,test)

In [418]:
import math
def fix_recency_of_branch_txn(row):
    if(math.isnan(row["Recency_of_BRANCH_TXN"]) or  row["Recency_of_BRANCH_TXN"] == 0):
        
        days = 210
        if(row["BRANCH_C_prev1"] > 0 or row["BRANCH_D_prev1"]>0):
            days = 30
        elif(row["BRANCH_C_prev2"] > 0 or row["BRANCH_D_prev2"]>0):
            days = 60
        elif(row["BRANCH_C_prev3"] > 0 or row["BRANCH_D_prev3"]>0):
            days = 90
        elif(row["BRANCH_C_prev4"] > 0 or row["BRANCH_D_prev4"]>0):
            days = 120
        elif(row["BRANCH_C_prev5"] > 0 or row["BRANCH_D_prev5"]>0):
            days = 150
        elif(row["BRANCH_C_prev6"] > 0 or row["BRANCH_D_prev6"]>0):
            days = 180
        if(row["Recency_of_BRANCH_TXN"] == 0 and days < 90):
            return 0
            
        return days
    else:
        return row["Recency_of_BRANCH_TXN"]
    
train["Recency_of_BRANCH_TXN_NEW"] = train.loc[:,:].apply(fix_recency_of_branch_txn,axis=1)

def fix_recency_of_credit_txn(row):
    if(math.isnan(row["Recency_of_CR_TXN"]) or row["Recency_of_CR_TXN"] == 0):
        days = 210
        if(row["C_prev1"] > 0 ):
            days = 30
        elif(row["C_prev2"] > 0):
            days = 60
        elif(row["C_prev3"] > 0 ):
            days = 90
        elif(row["C_prev4"] > 0 ):
            days = 120
        elif(row["C_prev5"] > 0 ):
            days = 150
        elif(row["C_prev6"] > 0 ):
            days = 180
        if(row["Recency_of_CR_TXN"] == 0 and days < 90):
            return 0
            
        return days
    else:
        return row["Recency_of_CR_TXN"]

train["Recency_of_CR_TXN_NEW"] = train.apply(fix_recency_of_credit_txn,axis=1)

def fix_recency_of_debit_txn(row):
    if(math.isnan(row["Recency_of_DR_TXN"]) or row["Recency_of_DR_TXN"] == 0):
        days = 210
        if(row["D_prev1"] > 0 ):
            days = 30
        elif(row["D_prev2"] > 0):
            days = 60
        elif(row["D_prev3"] > 0 ):
            days = 90
        elif(row["D_prev4"] > 0 ):
            days = 120
        elif(row["D_prev5"] > 0 ):
            days = 150
        elif(row["D_prev6"] > 0 ):
            days = 180
        if(row["Recency_of_DR_TXN"] == 0 and days < 90):
            return 0
            
        return days
    else:
        return row["Recency_of_DR_TXN"]

train["Recency_of_DR_TXN_NEW"] = train.apply(fix_recency_of_debit_txn,axis=1)


In [419]:
test["Recency_of_BRANCH_TXN_NEW"] = test.loc[:,:].apply(fix_recency_of_branch_txn,axis=1)
test["Recency_of_CR_TXN_NEW"] = test.apply(fix_recency_of_credit_txn,axis=1)
test["Recency_of_DR_TXN_NEW"] = test.apply(fix_recency_of_debit_txn,axis=1)

In [420]:
model_cols_list.append("Recency_of_BRANCH_TXN_NEW")
model_cols_list.append("Recency_of_CR_TXN_NEW")
model_cols_list.append("Recency_of_DR_TXN_NEW")

In [423]:
train[["I_NRV_PrevQ2","I_NRV_PrevQ1"]].describe()

,I_NRV_PrevQ2,I_NRV_PrevQ1
count,"300,000.00","300,000.00"
mean,"2,919,590.77","3,007,393.95"
std,"84,538,617.77","100,478,696.40"
min,"-2,520,748.02","100,000.00"
25%,"189,455.35","205,908.59"
50%,"397,296.38","411,897.96"
75%,"1,008,868.30","1,045,189.55"
max,"31,109,509,858.00","39,470,998,040.00"


In [436]:
test.loc[test["I_NRV_PrevQ2"] <= 0, ["I_NRV_PrevQ2","I_NRV_PrevQ1"]].describe()

,I_NRV_PrevQ2,I_NRV_PrevQ1
count,35.00,35.00
mean,"-287,819.59","2,468,720.01"
std,"1,430,898.31","7,372,282.59"
min,"-8,472,956.21","104,195.45"
25%,"-13,157.90","202,861.84"
50%,0.00,"371,504.94"
75%,0.00,"1,235,080.30"
max,0.00,"42,886,534.75"


In [432]:
train.loc[train["I_NRV_PrevQ2"] < 0,"Responders"].describe()

count                   0.00
mean                     nan
std                      nan
min                      nan
25%                      nan
50%                      nan
75%                      nan
max                      nan
Name: Responders, dtype: float64

In [437]:
test.loc[test["I_NRV_PrevQ2"] < 1,"I_NRV_PrevQ2"] = 1

In [447]:
# train["I_NRV_PrevQ_CV"] = 2*train["I_NRV_PrevQ1"] / (train["I_NRV_PrevQ1"] + train["I_NRV_PrevQ2"])

# test["I_NRV_PrevQ_CV"] = 2*test["I_NRV_PrevQ1"] / (test["I_NRV_PrevQ1"] + test["I_NRV_PrevQ2"])

train["I_NRV_PrevQ_CV"] = train["I_NRV_PrevQ1"] / ( train["I_NRV_PrevQ2"])

test["I_NRV_PrevQ_CV"] = test["I_NRV_PrevQ1"] / ( test["I_NRV_PrevQ2"])



In [441]:
test["I_NRV_PrevQ_CV"].describe()

count             200,000.00
mean                    1.05
std                     0.26
min                     0.00
25%                     0.95
50%                     1.01
75%                     1.12
max                     2.00
Name: I_NRV_PrevQ_CV, dtype: float64

In [442]:
model_cols_list.append("I_NRV_PrevQ_CV")


In [444]:
prem_closed_cols =["CC_PREM_CLOSED_PREVQ1","EDU_PREM_CLOSED_PREVQ1","TL_PREM_CLOSED_PREVQ1","CE_PREM_CLOSED_PREVQ1","CV_PREM_CLOSED_PREVQ1","TWL_PREM_CLOSED_PREVQ1","AGRI_PREM_CLOSED_PREVQ1","AL_CNC_PREM_CLOSED_PREVQ1","PL_PREM_CLOSED_PREVQ1","BL_PREM_CLOSED_PREVQ1","AL_PREM_CLOSED_PREVQ1","OTHER_LOANS_PREM_CLOSED_PREVQ1"]
train["prem_closed_loan"] = train[prem_closed_cols].notnull().sum(axis=1)
test["prem_closed_loan"] = test[prem_closed_cols].notnull().sum(axis=1)

In [445]:
model_cols_list.append("prem_closed_loan")